In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def kalkulasi_ips(lst_grd, lst_sks):
    # pastikan list sks tidak mengandung NaN
    # NaN berarti mahasiswa tersebut tidak mengambil
    # mata kuliah yang dimaksud
    temp = [i for i, e in enumerate(lst_grd) if str(e).lower() == 'nan']
    
    lst_grd = [element for (i,element) in enumerate(lst_grd) if i not in temp]
    lst_sks = [element for (i,element) in enumerate(lst_sks) if i not in temp]
    
    a = sum(x * y for x, y in zip(lst_grd, lst_sks))
    b = sum(lst_sks)
    
    return round(a / b, 2)

def kalkulasi_rerata(lst):
    return round(sum(lst) / len(lst), 2)

def kalkulasi_ips_tingkat_per_prodi(semester,
                                    prodi,
                                    angkatan):
    if prodi == 'Teknologi Rekayasa Perangkat Lunak (Tolikara)':
        df = pd.read_excel('./nilai/Rekap_Nilai_' + prodi.replace(' (Tolikara)','') + '_' + angkatan + '.xlsx',
                       engine = 'openpyxl',
                       header = [0, 1, 2, 3])
    else:
        df = pd.read_excel('./nilai/Rekap_Nilai_' + prodi + '_' + angkatan + '.xlsx',
                       engine = 'openpyxl',
                       header = [0, 1, 2, 3])
    
    df_grade = df.xs('Semester ' + str(semester), level=0, axis=1)
    df_grade = df_grade.iloc[: , :-4]
    
    cols_km = [str(c) for c in list(df_grade.columns.get_level_values(1))[:] 
              if c != 'Grade' and not 'Unnamed' in str(c)]
    cols_km.insert(0, "Nim")
    
    cols = [c for c in df_grade.columns if 'Grade' in c]
    df_grade = df_grade[cols]
    
    dft = pd.concat([df.xs('Nim', level=3, axis=1),
                     df_grade], axis=1)

    lst_tmp = []
    for idx, col in enumerate(dft):
        if idx == 0:
            lst_tmp.append(('SKS', 'Grade', cols_km[idx]))
        else:
            lst_tmp.append((col[0], col[1], cols_km[idx]))
    
    # sesuaikan nama kolom
    dft.columns = pd.MultiIndex.from_tuples(tuple(lst_tmp))
    
#     dft = dft.loc[dft.count(1) > dft.shape[1]/2, dft.count(0) > dft.shape[0]/2]
#     dft = dft.reset_index(drop=True)

    dict_grade = {'A' : 4.0,
                  'AB' : 3.5,
                  'B' : 3.0,
                  'BC' : 2.5,
                  'C' : 2.0,
                  'D' : 1.0,
                  'E' : 0,
                  'T' : np.nan
    }
    
    for i in dft:
        dft[i] = [dict_grade.get(item,item) for item in dft[i].tolist()]

    # penanganan khusus Prodi Teknologi Rekayasa Perangkat Lunak (Tolikara)
    if angkatan == str('2020'):
        if prodi == 'Teknologi Rekayasa Perangkat Lunak (Tolikara)':
            dft = dft[:25]
            dft = dft.dropna(axis=1)
        elif prodi == 'Teknologi Rekayasa Perangkat Lunak':
            dft = dft[25:]
    
    # hapus baris data apabila terdapat 50% nilai 0 
    mask = (dft.iloc[:, 1:] == float(0)).sum(1) >= len(dft.columns[1:]) / 2
    dft = dft[~mask]
    
    # hapus setiap kolom yang semua nilainya NaN
    dft = dft.dropna(how='all', axis=1)
    
    # hapus setiap baris yang semua nilainya NaN
    dft = dft.dropna(how='all', subset=dft.columns[1:])
    
    # reset index
    dft = dft.reset_index(drop=True)
    
    # list sks
    lst_sks = list(dft.columns.get_level_values(0))[1:]

    # list sks
    lst_tmp =[]
    for i in dft.itertuples():
        lst_grd = list(i[2:])
        kalkulasi_ips(lst_grd, lst_sks)
        lst_tmp.append(kalkulasi_ips(lst_grd, lst_sks))
    
#     display(dft)
    return kalkulasi_rerata(lst_tmp)

In [ ]:
def main(tahun_ajaran):
    aw, ak = tahun_ajaran[:len(tahun_ajaran)//2], tahun_ajaran[len(tahun_ajaran)//2:]
    aw = int("20" + aw)
    ak = int("20" + ak)

    dict_info = {}
    tingkat = 4
    semester = 8
    for i in range(aw-3, aw+1): 
        dict_info.update({str(tingkat) : {'Angkatan' : i ,
                          'Semester' : semester}})
        tingkat -= 1
        semester -= 2
        
    prodi = ['Teknologi Informasi',
             'Teknologi Komputer',
             'Teknologi Rekayasa Perangkat Lunak',
             'Teknologi Rekayasa Perangkat Lunak (Tolikara)',
             'Informatika',
             'Sistem Informasi',
             'Teknik Elektro',
             'Teknik Bioproses',
             'Manajemen Rekayasa']

    list_strata = ['D3', 'D3', 'D4', 'D4',
                   'S1', 'S1', 'S1',
                   'S1', 'S1']
    
    df_hasil = pd.DataFrame(columns=['Strata',
                                     'Prodi',
                                     'Tingkat',
                                     'Angkatan',
                                     'Semester'])

    for idx, ele in enumerate(prodi):
        srata = list_strata[idx]
        for key in dict_info:
            to_append = [srata, ele, key, dict_info[key]['Angkatan'], dict_info[key]['Semester']]
            a_series = pd.Series(to_append, index = df_hasil.columns)
            df_hasil = df_hasil.append(a_series, ignore_index=True)

    df_hasil.drop(df_hasil[(df_hasil['Strata'] == 'D3')  & (df_hasil['Tingkat'] == '4')].index, inplace = True)
    df_hasil.drop(df_hasil[(df_hasil['Prodi'] == 'Teknologi Rekayasa Perangkat Lunak (Tolikara)')  & 
                           ((df_hasil['Tingkat'] == '4') |
                            (df_hasil['Tingkat'] == '3') |
                            (df_hasil['Tingkat'] == '2')
                           )].index, inplace = True)
    
    df_hasil_akhir = pd.DataFrame(columns=['Tahun Ajaran',
                                           'Program Studi',
                                           'Tingkat',
                                           'Rata-rata'])
    
    for i in df_hasil.itertuples():
        try:
            ips_tingkat = kalkulasi_ips_tingkat_per_prodi(i.Semester,
                                                          i.Prodi,
                                                          str(i.Angkatan))
            
            to_append = [tahun_ajaran[:2]+'/'+tahun_ajaran[2:],
                         i.Prodi,
                         i.Tingkat,
                         ips_tingkat]
            a_series = pd.Series(to_append, index = df_hasil_akhir.columns)
            df_hasil_akhir = df_hasil_akhir.append(a_series, ignore_index=True)
            
        except Exception as e:
            to_append = [tahun_ajaran[:2]+'/'+tahun_ajaran[2:],
                         i.Prodi,
                         i.Tingkat,
                         float(0)]
            a_series = pd.Series(to_append, index = df_hasil_akhir.columns)
            df_hasil_akhir = df_hasil_akhir.append(a_series, ignore_index=True)
    
    return df_hasil_akhir

In [ ]:
# tahun_ajaran = str(2021)

In [ ]:
df_res = main(str(2021)).append(main(str(1920)))

In [ ]:
display(df_res)

In [ ]:
df_res.to_excel("ip_output.xlsx") 

In [ ]:
# kalkulasi_ips_tingkat_per_prodi('2',
#                                 'Teknik Bioproses',
#                                 str('2020'))

---